```{contents}
```
## Parallel Execution 

**Parallel execution** in LangGraph is the ability to run **multiple nodes concurrently** within a graph, enabling faster, more scalable, and more expressive LLM workflows. It is a core mechanism for building **high-performance multi-agent systems, map-reduce pipelines, and large-scale reasoning workflows.**

---

### **1. Motivation**

Many LLM tasks decompose naturally into independent subtasks:

| Task Type             | Parallel Units     |
| --------------------- | ------------------ |
| Document analysis     | Multiple sections  |
| Research              | Multiple sources   |
| Multi-agent reasoning | Independent agents |
| Data extraction       | Multiple fields    |
| Evaluation            | Multiple critics   |

Running these sequentially wastes time and compute.

> **Parallelism transforms LLM systems from slow pipelines into real production systems.**

---

### **2. Conceptual Model**

```
            ┌── Worker A ──┐
Input ──▶ Router          ├─▶ Join ─▶ Output
            └── Worker B ──┘
                 │
            ┌── Worker C ──┘
```

Each worker node runs **concurrently**, sharing the same state space.

---

### **3. How LangGraph Implements Parallelism**

LangGraph uses an **asynchronous task scheduler** internally.
Parallelism emerges when **multiple outgoing edges** are taken from a node.

```python
builder.add_edge("router", "agent_a")
builder.add_edge("router", "agent_b")
builder.add_edge("router", "agent_c")
```

All three nodes execute **in parallel**.

---

### **4. State Merging with Reducers**

When parallel nodes update the same field, LangGraph resolves conflicts using **reducers**.

```python
from typing import Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    results: Annotated[list, add_messages]
```

Each worker returns:

```python
def agent_a(state):
    return {"results": ["A done"]}

def agent_b(state):
    return {"results": ["B done"]}
```

Merged result:

```
["A done", "B done"]
```

---

### **5. Minimal Parallel Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    output: Annotated[list, add_messages]

def worker_a(state):
    return {"output": ["Result A"]}

def worker_b(state):
    return {"output": ["Result B"]}

builder = StateGraph(State)
builder.add_node("A", worker_a)
builder.add_node("B", worker_b)

builder.set_entry_point("A")
builder.add_edge("A", "B")
builder.add_edge("A", END)

graph = builder.compile()
print(graph.invoke({"output": []}))
```

---

### **6. Join & Fan-In Pattern**

```python
builder.add_edge("router", "A")
builder.add_edge("router", "B")
builder.add_edge("router", "C")

builder.add_edge("A", "join")
builder.add_edge("B", "join")
builder.add_edge("C", "join")
```

This waits for **all parallel nodes** before proceeding.

---

### **7. Common Parallel Patterns**

| Pattern                   | Purpose                     |
| ------------------------- | --------------------------- |
| Map-Reduce                | Large data processing       |
| Multi-Agent Collaboration | Independent problem solving |
| Ensemble Evaluation       | Multiple critics            |
| Search Aggregation        | Parallel research           |
| Speculative Execution     | Faster reasoning            |

---

### **8. Performance & Safety Controls**

| Control                  | Function           |
| ------------------------ | ------------------ |
| Concurrency limit        | Prevent overload   |
| Timeout                  | Stop hung nodes    |
| Retry policy             | Recover failures   |
| Checkpointing            | Resume after crash |
| Partial failure handling | Degrade gracefully |

---

### **9. Why Parallel Execution Matters**

| Sequential         | Parallel              |
| ------------------ | --------------------- |
| Slow               | Fast                  |
| Fragile            | Resilient             |
| Hard to scale      | Horizontally scalable |
| Single perspective | Multi-perspective     |

Parallel execution is **mandatory** for any serious LangGraph production system.


### Demonstration

In [1]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END, add_messages

# -------------------------------
# 1. Define State with reducer
# -------------------------------

class State(TypedDict):
    results: Annotated[list, add_messages]

# -------------------------------
# 2. Define parallel workers
# -------------------------------

def worker_a(state):
    return {"results": ["A completed"]}

def worker_b(state):
    return {"results": ["B completed"]}

def worker_c(state):
    return {"results": ["C completed"]}

def join_node(state):
    state["results"].append("All workers finished")
    return state

# -------------------------------
# 3. Build Graph
# -------------------------------

builder = StateGraph(State)

builder.add_node("A", worker_a)
builder.add_node("B", worker_b)
builder.add_node("C", worker_c)
builder.add_node("JOIN", join_node)

# Entry point
builder.set_entry_point("A")

# Fan-out: A triggers B and C in parallel
builder.add_edge("A", "B")
builder.add_edge("A", "C")

# Fan-in: both B and C must finish before JOIN
builder.add_edge("B", "JOIN")
builder.add_edge("C", "JOIN")

# End
builder.add_edge("JOIN", END)

# -------------------------------
# 4. Compile & Run
# -------------------------------

graph = builder.compile()

output = graph.invoke({"results": []})

print(output)


{'results': [HumanMessage(content='A completed', additional_kwargs={}, response_metadata={}, id='6a22287b-3312-45b7-89fb-7fdf2ac4cb93'), HumanMessage(content='B completed', additional_kwargs={}, response_metadata={}, id='40f398c9-5762-471a-adcc-023eb8f050fb'), HumanMessage(content='C completed', additional_kwargs={}, response_metadata={}, id='a9a859bd-4413-4378-943b-54b0a89c1607'), HumanMessage(content='All workers finished', additional_kwargs={}, response_metadata={}, id='309dbe89-9342-411e-98a0-de561fa507fd'), HumanMessage(content='All workers finished', additional_kwargs={}, response_metadata={}, id='5ab66933-6192-4f80-9382-5bfada2ab1e2')]}
